# A simplified n-body simulation in Python

For this tutorial, we will use the **gravitational N-body problem** in Python. We assume a 3D space of particles that interact with each other gravitationally and we will compute the system's kinetic and potential energy.  

We assume that each particle $ i $ has a mass $ m_i $, a position $ r_i=[x,y,z] $, and a velocity $ v_i = [vx_i,vy_i,vz_i] $. According to Newton's law, each particle will experience an acceleration $ a_i = G \sum_{j \neq i} m_j \frac{r_j - r_i}{|r_j - r_i|^3} $, where $ G $ is Newton's gravitational constant. 

We also assume that the system evolves in timesteps $\Delta t$, in a "kick-drift-kick" way. First, each particle receives a half-step kick and its velocity gets updated: $ v_i = v_i + \frac{\Delta t}{2} \times a_i$. Then, it updates its position using the updated velocity, $r_i = r_i + \Delta t \times v_i$. Finally the velocity gets updated again in the second half-step kick.

The total energy of the system is $E = KE + PE = \sum_i \frac{1}{2}m_i{v_i}^2 - \sum_{1\leq i\leq j\leq N} \frac{Gm_im_j}{|r_j - r_i|}$. 

For simplicity, we will focus on the kinetic energy (KE) only, therefore once the velocity is updated, we can compute the kinetic energy $ KE = \sum_i \frac{1}{2}m_i{v_i}^2 $.

*Note: The code for this tutorial has been adapted from https://github.com/pmocz/nbody-python, and a description of the assumptions for the relevant implementation can be found here: https://medium.com/swlh/create-your-own-n-body-simulation-with-matlab-22344954228e*

Let's start by setting up our problem.

#### Problem parameters

In [1]:
import numpy as np
import math

#Number of particles
N = 2000
#Start time
tStart = 0
#End time
tEnd = 1.
#Time step
dt = 0.2
#Number of steps
tSteps = math.ceil((tEnd - tStart)/dt)
#Newton's constant
G = 1.0
#Softening length to account for particles that are very close
softening = 0.1

#### Initializing particles (position, velocity,  mass) in the 3D-space
We will initialize matrices of size N with the position (3D), velocity (3D), and mass (1D) of each particle.

In [2]:
np.random.seed(17)

#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

#### Helper functions
We will create some helper functions here - we will optimize these later.

In [3]:
# Hadamard product
# Input: Matrix x (M x N), Matrix y (M x N)
# Output: Matrix (M x N)
def hadMatMat(x, y):
    return x * y

# Scaling of matrix x
# Input: Matrix x (M x N), scalar val
# Output: Matrix (M x N)
def sclMat(x, val):
    return val * x

# Matrix-Matrix product of matrices x,y
# Input: Matrix x (M x K), Matrix y (K x N)
# Output: Matrix (M x N)
def mulMatMat(x, y):
    return x @ y

# Computation of 1/|r_i - r_j|^3
# Input: Particle separations dx (N x N), dy (N x N), dz (N x N), softening (scalar)
# Output: inv_r3 (N x N)
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
    for i in range(inv_r3.shape[0]):
        for j in range(inv_r3.shape[1]):
            if (inv_r3[i,j] > 0):
                inv_r3[i,j] = inv_r3[i,j]**(-1.5)
    return inv_r3

# Pairwise particle separations r_j - r_i per dimension
# Input: Particle position in single dimension (N x 1)
# Output: matrix of separations N x N
def particleSeps(v):
    return v.T - v

# Computation of acceleration per dimension
# Input: velocity (N x 1), inv_r3 (N x N), mass (N x 1), G (scalar)
# Output: acceleration (N x 1)
def computeAcc(dv, inv_r3, mass, G): 
    av = hadMatMat(dv, inv_r3)
    av = sclMat(av, G)
    av = mulMatMat(av, mass)
    return av

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    # 1/r^3 for all particle separations
    inv_r3 = computeInvR3(dx, dy, dz, softening)
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a
    
# Computation of kinetic energy KE
# Input: positions (N x 3), mass (N x 1), velocity (N x 3), G (scalar)
# Output: KE (scalar)
def getEnergy(mass, vel):
    # Compute kinetic energy
    KE = 0.5 * np.sum(mass * vel**2)
    return KE

#### Main n-body loop
Let's put it all together in the main n-body loop.

In [4]:
def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

#### Run n-body
Now we can run our first N-body simulation.

In [5]:
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)
print("Kinetic Energy: ", KE)

Kinetic Energy:  110.38075827435034


## Exercise 1: Profile the N-body code

We will now profile the N-body given above, using Python `cProfile` and `pstats`.  

In [6]:
import cProfile, pstats

##TODO: Add a cProfile profiler 
profiler = cProfile.Profile()
##TODO: Start the profiler
profiler.enable()
#Initializing particles position, mass, and velocity
np.random.seed(17)
#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

KE = nbody(mass, pos, vel, dt, tSteps, G, softening)


##TODO: Stop the profiler
profiler.disable()
##TODO: Print profiler statistics (top 10) based on the tottime
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats(10)
print("Kinetic Energy: ", KE)

         383 function calls in 8.345 seconds

   Ordered by: internal time
   List reduced from 49 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    7.738    1.290    7.738    1.290 /tmp/ipykernel_2412/917823000.py:22(computeInvR3)
       18    0.213    0.012    0.213    0.012 /tmp/ipykernel_2412/917823000.py:33(particleSeps)
       18    0.189    0.010    0.189    0.010 /tmp/ipykernel_2412/917823000.py:4(hadMatMat)
       18    0.168    0.009    0.168    0.009 /tmp/ipykernel_2412/917823000.py:10(sclMat)
       18    0.023    0.001    0.023    0.001 /tmp/ipykernel_2412/917823000.py:16(mulMatMat)
       18    0.010    0.001    0.389    0.022 /tmp/ipykernel_2412/917823000.py:39(computeAcc)
        1    0.003    0.003    8.344    8.344 /tmp/ipykernel_2412/3995771557.py:1(nbody)
        2    0.001    0.000    0.001    0.000 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
        6    0.000    0.000    0.000    

### *Important note:* In iPython, we can use `IPython magic` to time the execution of a cell or a Python statement or expression. 

We can use `%%time` to measure the execution time of a cell, or `%%timeit`, to measure the execution time of a function in a cell (IPython will perform a number of repetitions and report the average).

Here is an example:

In [7]:
%%time
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

CPU times: user 11 s, sys: 12.5 s, total: 23.5 s
Wall time: 8.05 s


## Exercise 2: Accelerate the slowest functions using Numba

The profile of N-body shows that many functions consume considerable amount of the total execution time and can therefore be optimized. We will now use Numba decorators, to see how much acceleration we can achieve through Numba compiler optimizations. Start with annotating the most time-consuming function. Then, proceed to annotate as many functions as needed with Numba decorators. Use the `nopython=True` mode.

In [8]:
##TODO: Use Numba to accelerate as many functions as possible
import numba

##TODO: Use Numba decorator if needed

# Hadamard product
# Input: Matrix x (M x N), Matrix y (M x N)
# Output: Matrix (M x N)
@numba.jit(nopython=True)
def hadMatMat(x, y):
    return x * y

##TODO: Use Numba decorator if needed

# Scaling of matrix x
# Input: Matrix x (M x N), scalar val
# Output: Matrix (M x N)
@numba.jit(nopython=True)
def sclMat(x, val):
    return val * x

##TODO: Use Numba decorator if needed

# Matrix-Matrix product of matrices x,y
# Input: Matrix x (M x K), Matrix y (K x N)
# Output: Matrix (M x N)
@numba.jit(nopython=True)
def mulMatMat(x, y):
    return x @ y

##TODO: Use Numba decorator if needed

# Computation of 1/|r_i - r_j|^3
# Input: Particle separations dx (N x N), dy (N x N), dz (N x N), softening (scalar)
# Output: inv_r3 (N x N)
@numba.jit(nopython=True)
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
    for i in range(inv_r3.shape[0]):
        for j in range(inv_r3.shape[1]):
            if (inv_r3[i,j] > 0):
                inv_r3[i,j] = inv_r3[i,j]**(-1.5)
    return inv_r3

##TODO: Use Numba decorator if needed

# Pairwise particle separations r_j - r_i per dimension
# Input: Particle position in single dimension (1 x N)
# Output: matrix of separations N x N
@numba.jit(nopython=True)
def particleSeps(v):
    return v.T - v

##TODO: Use Numba decorator if needed

# Computation of acceleration per dimension
# Input: velocity (N x 1), inv_r3 (N x N), mass (1 x N), G (scalar)
# Output: acceleration (N x 1)
@numba.jit(nopython=True)
def computeAcc(dv, inv_r3, mass, G): 
    av = hadMatMat(dv, inv_r3)
    av = sclMat(av, G)
    av = mulMatMat(av, mass)
    return av

##TODO: Use Numba decorator if needed

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
@numba.jit(nopython=True)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    # 1/r^3 for all particle separations
    inv_r3 = computeInvR3(dx, dy, dz, softening)
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a
    
##TODO: Use Numba decorator if needed

# Computation of kinetic energy KE
# Input: positions (N x 3), mass (N x 1), velocity (N x 3), G (scalar)
# Output: KE (scalar)
@numba.jit(nopython=True)
def getEnergy(mass, vel):
    # Compute kinetic energy
    KE = 0.5 * np.sum(mass * vel**2)
    return KE

##TODO: Use Numba decorator if needed

# Computation of kinetic energy KE
# Input: positions (N x 3), mass (N x 1), velocity (N x 3), G (scalar)
# Output: KE (scalar)
@numba.jit(nopython=True)
def getEnergy(mass, vel):
    # Compute kinetic energy
    KE = 0.5 * np.sum(mass * vel**2)
    return KE

##TODO: Use Numba decorator if needed
@numba.jit(nopython=True)
def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

Let's measure the execution time using Numba.

In [9]:
#Initialize particles position, mass, and velocity
np.random.seed(17)
#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

In [10]:
%%timeit
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

2.06 s ± 522 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
print("Kinetic Energy: ", KE)

Kinetic Energy:  59.76978330223841


### *Important note:* If we try to use the Python profiler with Numba, we will not be able to get useful information about the most-time consuming functions anymore, as there are now dispatched to / handled by Numba.

## Exercise 3: Parallelize the `computeInvR3` function using Numba

We will now parallelize a function using Numba. We will try to use both implicit and explicit parallelization. For the function `computeInvR3`, modify your Numba annotation, to include the `parallel=True` mode. This is implicit parallelization. Then you can try using `numba.prange` to mark the function loops as parallelizable. 

In [12]:
##TODO: Parallelize the computeInvR3 function using Numba

##TODO: add a Numba decorator
@numba.jit(nopython=True, parallel=True)
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
    ##TODO: substitute range with numba.prange
    for i in numba.prange(inv_r3.shape[0]):
        for j in numba.prange(inv_r3.shape[1]):
            if (inv_r3[i,j] > 0):
                inv_r3[i,j] = inv_r3[i,j]**(-1.5)
    return inv_r3

#TODO: add a Numba decorator

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
@numba.jit(nopython=True)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    # 1/r^3 for all particle separations
    inv_r3 = computeInvR3(dx, dy, dz, softening)
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a


#TODO: add a Numba decorator
@numba.jit(nopython=True)
def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

Before we run N-body, let's discover the number of threads on our CPU.

In [13]:
print("Threads: ", numba.get_num_threads())

Threads:  16


In [14]:
#Initialize particles position, mass, and velocity
np.random.seed(17)
#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

In [15]:
%%timeit
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

2.01 s ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
print(KE)

59.76978330223841


## Exercise 4: Accelerate the `computeInvR3` function on the GPU using Numba

We will use the GPU to accelerate the `computeInvR3` function. Before we begin, we will first rewrite this function to make it more friendly to the CUDA programming style. 

In [17]:
from numba import cuda
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = np.empty((dx.shape[0], dx.shape[1]), dtype = np.float32)
    for i in range(dx.shape[0]):
        for j in range(dx.shape[1]):
            inv_r3[i,j] = dx[i,j]**2 + dy[i,j]**2 + dz[i,j]**2 + softening**2
            if (inv_r3[i,j] > 0):
                inv_r3[i,j] = inv_r3[i,j]**(-1.5)
    return inv_r3

### Exercise 4.1:  Re-write the function `computeInvR3` with the correct arguments and annotate it with Numba
Remember that CUDA Numba functions must not return any values. 

In [18]:
import os
# os.environ['CUDA_HOME']='/chalmers/sw/sup64/cuda_toolkit-11.2.2'

from numba import cuda

##TODO: Annotate this function with Numba
##TODO: Change the function arguments
@numba.cuda.jit
def computeInvR3(dx, dy, dz, softening, inv_r3):
    ##TODO: Do we really need this initialization?    
    # inv_r3 = np.empty((dx.shape[0], dx.shape[1]), dtype = np.float32)
    for i in range(dx.shape[0]):
        for j in range(dx.shape[1]):
            inv_r3[i,j] = dx[i,j]**2 + dy[i,j]**2 + dz[i,j]**2 + softening**2
            if (inv_r3[i,j] > 0):
                inv_r3[i,j] = inv_r3[i,j]**(-1.5)
    return

### Exercise 4.2:  Distribute the work in the function to threads and blocks of threads
Since all the matrices used in the function are 2D (`dx`, `dy`, `dz` and `inv_r3`), we will assume a 2D grid of threads and 2D blocks of threads. 

In [163]:
##TODO: Copy the computeInvR3 function from the previous cell 
##TODO: ...and rewrite it to make it CUDA-friendly
@cuda.jit(device=True)
def my_pow(x, y):
    return x ** y

@numba.cuda.jit
def computeInvR3(dx, dy, dz, softening, inv_r3):
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    if (i >= dx.shape[0] or j >= dx.shape[1]):
        return
    inv_r3[i, j] = my_pow(dx[i, j], 2) + my_pow(dy[i, j], 2) + my_pow(dz[i, j], 2) + my_pow(softening[0], 2)
    # inv_r3[i,j] = dx[i,j]**2 + dy[i,j]**2 + dz[i,j]**2 + softening**2
    if (inv_r3[i,j] > 0):
        inv_r3[i,j] = my_pow(inv_r3[i,j], (-1.5))
    return

### Exercise 4.3: Copy the input matrices from the CPU to the GPU and the results from the GPU to the CPU
Remember that, when the function is called, data is initially allocated on the CPU. To run the function on the GPU, we need to transfer all data (`dx`, `dy`, `dz`) to the GPU. Once the function is complete, we also need to copy the result `inv_r3` from the GPU to the CPU.

### Exercise 4.4: Allocate space on the GPU for the result of the function
The result of the operation `inv_r3` is produced on the GPU. Before we call the function, we need to allocate space for this matrix on the GPU. 

### Exercise 4.5: Launch the kernel
Before we launch the kernel, we need to define the number of threads per block (total maximum = 1024 for this GPU) and the number of blocks in the grid. We can then launch the kernel by calling the GPU-accelerated function. We assume that the thread blocks will be 2-dimensional and of size (32,32). Define the number of blocks in the grid and launch the kernel.

In [164]:

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    ##TODO: Copy dx, dy, dz to the GPU
    ##TODO: Uncomment and fill in the next three lines
    gpu_dx = cuda.to_device(dx)
    gpu_dy = cuda.to_device(dy)
    gpu_dz = cuda.to_device(dz)
    gpu_softening = cuda.to_device([softening])
    
    ##TODO: Allocate space for inv_r3 on the GPU 
    ##TODO: Uncomment and fill in the next line
    # inv_r3 = np.empty((dx.shape[0], dx.shape[1]), dtype = np.float32)
    inv_r3_gpu = cuda.device_array_like(dx)
    
    threadsperblock = (32,32)
    #TODO: Define the number of blocks in the grid
    ##TODO: Define the number of blocks in the grid
    ##TODO: Uncomment and fill in the next line
    blockspergrid = (math.ceil(x.shape[0] / threadsperblock[0]),
                     math.ceil(x.shape[1] / threadsperblock[1])
                    )
    
    ##TODO: Launch the kernel `computeInvR3`
    ##TODO: Uncomment and fill in the next line
    #computeInvR3
    computeInvR3[blockspergrid, threadsperblock](gpu_dx, gpu_dy, gpu_dz, gpu_softening, inv_r3_gpu)
    ##TODO: Copy inv_r3 back to the CPU
    ##TODO: Uncomment and fill in the next line
    inv_r3 = inv_r3_gpu.copy_to_host()
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a

def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

In [165]:
import cupy
cupy.ones((2,2)).device
a = np.array(0.1)
a

array(0.1)

We are all set! Before we run nbody, let's discover our GPUs. 

In [166]:
print("CUDA-enabled GPUs:", cuda.gpus)

CUDA-enabled GPUs: <Managed Device 0>


In [167]:
#Initialize particles position, mass, and velocity
np.random.seed(17)
#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

In [170]:
%%timeit
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

790 ms ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [171]:
print(KE)

29.789274199762946


## Exercise 5: Accelerate N-body on the GPU using CuPy

To accelerate the N-body kernel with CuPy, we will start with re-writing the application to use NumPy functions. 

In [ ]:
# Hadamard product
# Input: Matrix x (M x N), Matrix y (M x N)
# Output: Matrix (M x N)
def hadMatMat(x, y):
    return np.multiply(x,y)

# Scaling of matrix x
# Input: Matrix x (M x N), scalar val
# Output: Matrix (M x N)
def sclMat(x, val):
    return np.multiply(x,val)

# Matrix-Matrix product of matrices x,y
# Input: Matrix x (M x K), Matrix y (K x N)
# Output: Matrix (M x N)
def mulMatMat(x, y):
    return np.matmul(x,y)

# Computation of 1/|r_i - r_j|^3
# Input: Particle separations dx (N x N), dy (N x N), dz (N x N), softening (scalar)
# Output: inv_r3 (N x N)
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = np.add(np.add(np.add(dx**2, dy**2),dz**2),softening**2)
    inv_r3[inv_r3>0] = inv_r3[inv_r3 > 0]**(-1.5) 
    return inv_r3

# Pairwise particle separations r_j - r_i per dimension
# Input: Particle position in single dimension (N x 1)
# Output: matrix of separations N x N
def particleSeps(v):
    return np.subtract(np.transpose(v),v)

# Computation of acceleration per dimension
# Input: velocity (N x 1), inv_r3 (N x N), mass (N x 1), G (scalar)
# Output: acceleration (N x 1)
def computeAcc(dv, inv_r3, mass, G): 
    av = hadMatMat(dv, inv_r3)
    av = sclMat(av, G)
    av = mulMatMat(av, mass)
    return av

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    # 1/r^3 for all particle separations
    inv_r3 = computeInvR3(dx, dy, dz, softening)
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a
    
# Computation of kinetic energy KE
# Input: positions (N x 3), mass (N x 1), velocity (N x 3), G (scalar)
# Output: KE (scalar)
def getEnergy(mass, vel):
    # Compute kinetic energy
    KE = 0.5 * np.sum(mass * vel**2)
    return KE

def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

We can now run the code again and see how using Numpy compares to using Numba. You can also use `cProfile` to analyze the execution time breakdown per function, as in Exercise #1.

In [ ]:
#Initialize particles position, mass, and velocity

np.random.seed(17)

#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

In [ ]:
%%timeit
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

In [ ]:
print(KE)

Let's now replace `numpy` with `cupy` and run all functions on the GPU.

In [ ]:
#TODO: Import cupy


# Hadamard product
# Input: Matrix x (M x N), Matrix y (M x N)
# Output: Matrix (M x N)
def hadMatMat(x, y):
    return np.multiply(x,y)

# Scaling of matrix x
# Input: Matrix x (M x N), scalar val
# Output: Matrix (M x N)
def sclMat(x, val):
    return np.multiply(x,val)

# Matrix-Matrix product of matrices x,y
# Input: Matrix x (M x K), Matrix y (K x N)
# Output: Matrix (M x N)
def mulMatMat(x, y):
    return np.matmul(x,y)

# Computation of 1/|r_i - r_j|^3
# Input: Particle separations dx (N x N), dy (N x N), dz (N x N), softening (scalar)
# Output: inv_r3 (N x N)
def computeInvR3(dx, dy, dz, softening):
    inv_r3 = np.add(np.add(np.add(dx**2, dy**2),dz**2),softening**2)
    inv_r3[inv_r3>0] = inv_r3[inv_r3 > 0]**(-1.5) 
    return inv_r3

# Pairwise particle separations r_j - r_i per dimension
# Input: Particle position in single dimension (N x 1)
# Output: matrix of separations N x N
def particleSeps(v):
    return np.subtract(np.transpose(v),v)

# Computation of acceleration per dimension
# Input: velocity (N x 1), inv_r3 (N x N), mass (N x 1), G (scalar)
# Output: acceleration (N x 1)
def computeAcc(dv, inv_r3, mass, G): 
    av = hadMatMat(dv, inv_r3)
    av = sclMat(av, G)
    av = mulMatMat(av, mass)
    return av

# Computation of acceleration 
# Input: positions (N x 3), mass (N x 1), G (scalar), softening (scalar)
# Output: accelerations (N x 3)
def getAcc(pos, mass, G, softening):
    # Positions for all particles
    x = pos[:, 0:1]
    y = pos[:, 1:2]
    z = pos[:, 2:3]
    
    # Particle separations
    dx = particleSeps(x)
    dy = particleSeps(y)
    dz = particleSeps(z)
    
    # 1/r^3 for all particle separations
    inv_r3 = computeInvR3(dx, dy, dz, softening)
    
    # Acceleration components per dimension
    ax = computeAcc(dx, inv_r3, mass, G)
    ay = computeAcc(dy, inv_r3, mass, G)
    az = computeAcc(dz, inv_r3, mass, G)
    
    # Packing acceleration components
    a = np.hstack((ax, ay, az))
    
    return a
    
# Computation of kinetic energy KE
# Input: positions (N x 3), mass (N x 1), velocity (N x 3), G (scalar)
# Output: KE (scalar)
def getEnergy(mass, vel):
    # Compute kinetic energy
    KE = 0.5 * np.sum(mass * vel**2)
    return KE

def nbody(mass, pos, vel, dt, tSteps, G, softening):
    # Convert to center-of-mass frame
    vel -= (np.sum(mass * vel) / (mass.shape[0] * vel.shape[1])) / (np.sum(mass) / (mass.shape[0] * mass.shape[1]))
    
    # Calculate initial gravitational accelerations
    acc = getAcc(pos, mass, G, softening)
    
    # Calculate initial energy
    KE = getEnergy(mass, vel)
    
    for i in range(tSteps):
        # First kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0
        
        # Drift -- Update position
        pos += vel * dt
        
        # Update acceleration
        acc = getAcc(pos, mass, G, softening)
        
        # Second kick (1/2) -- Compute velocity
        vel += acc * dt / 2.0

        KE = getEnergy(mass, vel)
        
    return KE

We will need to re-initialize our arrays with `cupy`, not just for correctness, but also to allocate them on the GPU instead of the CPU.

In [ ]:
np.random.seed(17)

#Position
pos = np.random.randn(N, 3)
#Velocity
vel = np.random.randn(N, 3)
#Mass
mass = 20. * np.ones((N, 1)) / N

In [ ]:
%%timeit
KE = nbody(mass, pos, vel, dt, tSteps, G, softening)

In [ ]:
print(KE)

## Exercise 6: Profile N-body on the GPU using CuPy

Having run our GPU-accelerated N-Body code, we can profile it using CuPy. 

In [ ]:
##TODO: Import the `benchmark` function from CuPy

##TODO: Use the benchmark function to profile nbody for 10 repetitions


## Exercise 7: Parallelize N-body for multiple processes using MPI (`mpi4py`)

So far, we have been running on a single node (single CPU with multiple cores, and single GPU). However, if we want to scale our application out to multiple nodes, we need to parallelize the application with the appropriate programming model. We will be looking at parallelization across distributed nodes (i.e. a cluster with multiple interconnected nodes), and we will use MPI.

For this exercise, we will distribute the *N* particles of the simulation to *P* processes, which will work together to compute the total kinetic energy *KE* of the system at each time step. 

As a reminder, we initialize three variables per particle, its mass *m*, its position *pos* and its velocity *v*. 

Below there is a template of the parallel function for N-body `parallel_nbody`, run with MPI with 4 processes. It currently only collects the size of the communicator (number of processes) and the rank (ID) of each process. You can run it and notice how *each* process executes the same piece of code, i.e. the `parallel_nbody` function.

In [ ]:
import ipyparallel as ipp
from mpi4py import MPI

def parallel_nbody():
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()
    return f"Size: {size} Rank: {rank}"
    
with ipp.Cluster(engines='mpi', n = 4) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(parallel_nbody)
    print("\n".join(r))

### Exercise 7.1: Distribute particles to processes with MPI

The first step towards parallelization will be to distribute the particles, and in particular, their associated variables for mass, position, and velocity to all processes. For this purpose, the process with *rank = 0* will initialize the relevant arrays and will then distribute them to all processes. Your task is to determine the appropriate function, to distribute the particles. We will modify the `parallel_nbody` function accordingly.

In [ ]:
import ipyparallel as ipp
from mpi4py import MPI

def parallel_nbody():
    import numpy as np
    import math

    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()
    #Number of particles
    N = 2000
    #Other parameters
    tStart = 0
    tEnd = 1.
    dt = 0.2
    tSteps = math.ceil((tEnd - tStart)/dt)
    G = 1.0
    softening = 0.1

    pos = None
    vel = None
    mass = None
    if (rank == 0):
        #IMPORTANT: We initialize all arrays as 1D, and then reshape
        #IMPORTANT: This is to make sure that memory is contiguous
        np.random.seed(17)
        #Position
        pos = np.random.randn(N * 3)
        #Velocity
        vel = np.random.randn(N * 3)
        #Mass
        mass = (20./ N) * np.ones(N) 


    #All processes initialize empty arrays of N / size particles
    #IMPORTANT: MPI needs to send/receive from flattened arrays (2D->1D)
    #IMPORTANT: We will initialize the arrays as 1D arrays and will reshape later
    pos_local = np.zeros((math.ceil(N/size) * 3), dtype = np.float64)
    vel_local = np.zeros((math.ceil(N/size) * 3), dtype = np.float64)
    mass_local = np.zeros((math.ceil(N/size) * 1), dtype = np.float64)
        
    
    #TODO: Replace comm.Dummy with the correct MPI function to distribute chunks of pos, vel, mass to all processes 
    #TODO: Use new arrays pos_local, vel_local, mass_local
    comm.Dummy(pos, pos_local, root=0)
    comm.Dummy(vel, vel_local, root=0)
    comm.Dummy(mass, mass_local, root=0)
    
    if (rank == 0):
        pos = pos.reshape(N, 3)
        vel = vel.reshape(N, 3)
        mass = mass.reshape(N, 1)

    pos_local = pos_local.reshape(math.ceil(N/size),3)
    vel_local = vel_local.reshape(math.ceil(N/size),3)
    mass_local = mass_local.reshape(math.ceil(N/size),1)
    
    return 
    
with ipp.Cluster(engines='MPI', n = 4) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(parallel_nbody)
    print("\n".join(r))

### Exercise 7.2: Understand parallelism in the `nbody` function and use the appropriate MPI functions

If we call `nbody` from the `parallel_nbody` function, all processes will run the N-body code on the particles they own. However, there are two places within the application where processes need to collaborate.  
1. In the `getAcc` function, where we calculate the acceleration, we first need to calculate all the particle separations, i.e. all the interactions between all particles in the simulation. This calculation depends on positions `pos`. Therefore, before each process calls this function, we need to ensure that all processes get the updated positions of all particles from all other processes. Which function is required? 
2. In the `getAcc` function, the calculation of the acceleration depends on the masses of all particles in the simulation. Therefore, according to our proposed parallelization scheme, its process computes a partial acceleration for all particles, and needs to accumulate all the accelerations for all particles, in order to collect the acceleration for its own particles. This part is given, and also affects the `computeAcc` function. Which function is used?
2. In the `nbody` function, each process calculates the kinetic energy KE of the particles that are assigned to it. However, eventually, we want to accumulate the kinetic energy from all processes and end up with all processes having the summed, global value for the whole system kinetic energy. Which function is required? 

Below, we will modify the two functions accordingly. 

In [ ]:
import ipyparallel as ipp
from mpi4py import MPI
import numpy as np


def parallel_nbody():
    import numpy as np
    import math

    
    def hadMatMat(x, y):
        return np.multiply(x,y)


    def sclMat(x, val):
        return np.multiply(x,val)

    def mulMatMat(x, y):
        return np.matmul(x,y)

    def computeInvR3(dx, dy, dz, softening):
        inv_r3 = np.add(np.add(np.add(dx**2, dy**2),dz**2),softening**2)
        inv_r3[inv_r3>0] = inv_r3[inv_r3 > 0]**(-1.5) 
        return inv_r3

    def particleSeps(v):
        return np.subtract(np.transpose(v),v)

    def computeAcc(dv, inv_r3, local_mass, G, comm): 
        rank = comm.Get_rank()
        size = comm.Get_size()
        N = dv.shape[0]
        av = hadMatMat(dv, inv_r3)
        av = sclMat(av, G)
        av = mulMatMat(av[:,rank * int(N/size):(rank+1) * int(N/size)], local_mass)
        return av

    def getAcc(local_pos, local_mass, G, softening, comm):
        rank = comm.Get_rank()
        size = comm.Get_size()
        N = local_pos.shape[0] * size

        # Positions for all particles

        #This function gets local_pos as an argument, but pos is missing (all positions)
        #Collect all `local_pos` from all other processes in one array, `pos`
        
        #IMPORTANT: We need to create flat arrays (1D) 
        pos = np.zeros(N * 3).reshape(N, 3)
        #TODO: Replace comm.Dummy with the correct function
        comm.Dummy(local_pos, pos)
        
        x = pos[:, 0:1]
        y = pos[:, 1:2]
        z = pos[:, 2:3]
    
        # Particle separations
        dx = particleSeps(x)
        dy = particleSeps(y)
        dz = particleSeps(z)
    
        # 1/r^3 for all particle separations
        inv_r3 = computeInvR3(dx, dy, dz, softening)
    
        # Acceleration components per dimension
        partial_ax = computeAcc(dx, inv_r3, local_mass, G, comm)
        partial_ay = computeAcc(dy, inv_r3, local_mass, G, comm)
        partial_az = computeAcc(dz, inv_r3, local_mass, G, comm)
        
        ax = np.zeros(partial_ax.shape)
        ay = np.zeros(partial_ay.shape)
        az = np.zeros(partial_az.shape)
        
        comm.Allreduce(partial_ax, ax)
        comm.Allreduce(partial_ay, ay)
        comm.Allreduce(partial_az, ay)
        # Packing acceleration components
        a = np.hstack((ax[rank * int(N/size):(rank+1) * int(N/size), :], ay[rank * int(N/size):(rank+1) * int(N/size),:], az[rank * int(N/size):(rank+1) * int(N/size)]))
        return a
    
    def getEnergy(local_mass, local_vel):
        # Compute kinetic energy
        KE = 0.5 * np.sum(local_mass * local_vel**2)
        return KE

    def nbody(local_mass, local_pos, local_vel, dt, tSteps, G, softening, comm):
        # Convert to center-of-mass frame
        local_vel -= (np.sum(local_mass * local_vel) / (local_mass.shape[0] * local_vel.shape[1])) / (np.sum(local_mass) / (local_mass.shape[0] * local_mass.shape[1]))
        
        # Calculate initial gravitational accelerations
        local_acc = getAcc(local_pos, local_mass, G, softening, comm)
        print(local_acc.shape)
        # Calculate initial energy
        local_KE = getEnergy(local_mass, local_vel)
        
        for i in range(tSteps):
            # First kick (1/2) -- Compute velocity
            
            local_vel += local_acc * dt / 2.0
            
            # Drift -- Update position
            local_pos += local_vel * dt
        
            # Update acceleration
            local_acc = getAcc(local_pos, local_mass, G, softening, comm)
        
            # Second kick (1/2) -- Compute velocity
            local_vel += local_acc * dt / 2.0

            local_KE = getEnergy(local_mass, local_vel)
            
        #Return the "global KE": sum the local kinetic energies on all processes
        KE = np.zeros(1)
        #TODO: Replace comm.Dummy with the correct function
        comm.Dummy(sendbuf=[local_KE, MPI.DOUBLE], recvbuf=[KE, MPI.DOUBLE])
        return KE

    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()
    #Number of particles
    N = 2000
    #Other parameters
    tStart = 0
    tEnd = 1.
    dt = 0.2
    tSteps = math.ceil((tEnd - tStart)/dt)
    G = 1.0
    softening = 0.1

    pos = None
    vel = None
    mass = None
    if (rank == 0):
        np.random.seed(17)
        #Position
        pos = np.random.randn(N * 3)
        #Velocity
        vel = np.random.randn(N * 3)
        #Mass
        mass = (20./ N) * np.ones(N) 


    #All processes initialize empty arrays of N / size particles
    #IMPORTANT: MPI needs to send/receive from flattened arrays (2D->1D)
    #IMPORTANT: We will initialize the arrays as 1D arrays and will reshape later
    pos_local = np.zeros((math.ceil(N/size) * 3), dtype = np.float64)
    vel_local = np.zeros((math.ceil(N/size) * 3), dtype = np.float64)
    mass_local = np.zeros((math.ceil(N/size) * 1), dtype = np.float64)
        
    
    #TODO: Replace comm.Dummy with the correct MPI function to distribute chunks of pos, vel, mass to all processes 
    comm.Dummy(pos, pos_local, root=0)
    comm.Dummy(vel, vel_local, root=0)
    comm.Dummy(mass, mass_local, root=0)
    
    if (rank == 0):
        pos = pos.reshape(N, 3)
        vel = vel.reshape(N, 3)
        mass = mass.reshape(N, 1)

    pos_local = pos_local.reshape(math.ceil(N/size),3)
    vel_local = vel_local.reshape(math.ceil(N/size),3)
    mass_local = mass_local.reshape(math.ceil(N/size),1)
    
    #We can call the updated nbody function here
    KE = nbody(mass_local, pos_local, vel_local, dt, tSteps, G, softening, comm)
    return f'Rank {rank}, Energy {KE}'
    


Now let's put it all together:

In [ ]:
with ipp.Cluster(engines='MPI', n = 4) as rc:
    view = rc.broadcast_view()
    r = view.apply(parallel_nbody)
    print("\n".join(r))

# That's all, folks! #